# Import libraries

In [52]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import re
path_desktop = "/mnt/c/Users/matth/Desktop"

# Usefull functions

In [51]:
def to_snake_case(col):
    col = col.strip().lower()
    col = col.replace("/", "_")
    col = re.sub(r"[^\w\s]", "", col)      # remove punctuation (#, /, -, ())
    col = re.sub(r"\s+", "_", col)         # spaces → underscores
    col = re.sub(r"_+", "_", col)          # collapse multiple underscores
    # fix leading digits (e.g. 1st_assistent → first_assistent)
    col = re.sub(r"^1st_", "first_", col)
    col = re.sub(r"^2nd_", "second_", col)
    col = re.sub(r"^3nd_", "third", col)
    
    return col.strip("_")


def excel_time_to_minutes(x):
    if pd.isna(x):
        return np.nan
    if isinstance(x, dt.time):
        return x.hour * 60 + x.minute + x.second / 60
    try:
        return pd.to_timedelta(x).total_seconds() / 60
    except Exception:
        return np.nan

def to_analyze(df, cols):
    to_analyze = df[cols]
    path_desktop = "/mnt/c/Users/matth/Desktop"
    path_file = f"{path_desktop}/to_analyze.csv"
    to_analyze.to_csv(path_file, index=False)

In [3]:
base_dir ="../Nkhoma_data/Data"

In [4]:
# all the files
os.listdir(base_dir)

['old_theatre_books_clean.xlsx',
 'Theatre_Book-Database 2025-plain.xlsx',
 'old_theatre_books_clean.pkl',
 'Old Theatre Books.xlsx',
 '.ipynb_checkpoints',
 'Theatre_Book-Database 2022 Auswertung-Arbeitsversion.xlsx',
 'Theatre_Book-Database 2024 Auswertung-Arbeitsversion.xlsx',
 'theatre_book_database_2022_clean.pkl',
 'Theatre_Book-Database 2023 Auswertung-Arbeitsversion.xlsx',
 'theatre_book_database_2022_clean.xlsx']

# Lets clean Theatre_Book-Database 2022 Auswertung-Arbeitsversion.xlsx

In [5]:
file_to_clean = "Theatre_Book-Database 2022 Auswertung-Arbeitsversion.xlsx"
path = f"{base_dir}/{file_to_clean}"
df = pd.read_excel(path, engine="openpyxl")
df.head()

/home/mat2m10/.pyenv/versions/3.12.9/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Theatre Book #,Hospital ID #,DATE of Surgery,First Name,Last Name,Age (years),Sex,Village,Surgeon,1st Assistent/Instructor,...,Urgency,Surgery severity,ASA-Score,Year of birth,Operation time (minutes),Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,220001,NaN,2022-01-01,Elifa,Sumati,26.0,F,Nkhonde,Obs/Gyn,NaN,...,NaN,NaN,NaN,1997.0,00:00:00,NaN,Calculated: do not fill out,Fill out for every patient,Fill out for all PAACS cases,Fill out if possible
1,220002,NaN,2022-01-01,Siyatu,Isaac,27.0,F,Mozambique,Obs/Gyn,NaN,...,NaN,NaN,NaN,1996.0,00:00:00,NaN,NaN,NaN,NaN,NaN
2,220003,NaN,2022-01-02,Loness,Mapemphero,25.0,F,Chembe,Obs/Gyn,NaN,...,NaN,NaN,NaN,1998.0,00:00:00,NaN,NaN,NaN,NaN,NaN
3,220004,NaN,2022-01-03,Saizi,Nedson,48.0,M,Chilikumanda,Limbe,Caleb,...,Emergency,Major,ASA 3,1975.0,00:00:00,NaN,NaN,NaN,NaN,NaN
4,220005,NaN,2022-01-03,Beatrice,Hezekia,26.0,F,Mazengera,Obs/Gyn,NaN,...,NaN,NaN,NaN,1997.0,00:00:00,NaN,NaN,NaN,NaN,NaN


In [6]:
# Drop unnamed
df = df.loc[:, ~df.columns.str.match(r"^Unnamed")]
# Drop only NaN cols
df = df.dropna(axis=1, how="all")
# Rename cols
df.columns = [to_snake_case(c) for c in df.columns]

# coerce theatre_book to numeric (invalid entries → NaN)
df["theatre_book"] = pd.to_numeric(df["theatre_book"], errors="coerce")

# keep only rows with a valid theatre_book number
df = df.dropna(subset=["theatre_book"])

# optional: make it integer
df["theatre_book"] = df["theatre_book"].astype(int)

df.shape, df.tail()

# rewrite typos
df = df.rename(columns={
    "sarting_time": "starting_time",
    "asascore": "asa_score",
})

start_m = df["starting_time"].apply(excel_time_to_minutes)
finish_m = df["finishing_time"].apply(excel_time_to_minutes)

# raw difference
df["op_minutes_calc"] = finish_m - start_m

# handle surgeries crossing midnight (e.g., 23:50 -> 00:20)
df.loc[df["op_minutes_calc"] < 0, "op_minutes_calc"] += 24 * 60

# convert operation_time_minutes to numeric minutes (reuse earlier logic)
df["op_minutes_recorded"] = df["operation_time_minutes"].apply(excel_time_to_minutes)

df

,theatre_book,hospital_id,date_of_surgery,first_name,last_name,age_years,sex,village,surgeon,first_assistent_instructor,...,histology,starting_time,finishing_time,urgency,surgery_severity,asa_score,year_of_birth,operation_time_minutes,op_minutes_calc,op_minutes_recorded
0,220001,NaN,2022-01-01,Elifa,Sumati,26.0,F,Nkhonde,Obs/Gyn,NaN,...,No,NaN,NaN,NaN,NaN,NaN,1997.0,00:00:00,NaN,0.0
1,220002,NaN,2022-01-01,Siyatu,Isaac,27.0,F,Mozambique,Obs/Gyn,NaN,...,No,NaN,NaN,NaN,NaN,NaN,1996.0,00:00:00,NaN,0.0
2,220003,NaN,2022-01-02,Loness,Mapemphero,25.0,F,Chembe,Obs/Gyn,NaN,...,No,NaN,NaN,NaN,NaN,NaN,1998.0,00:00:00,NaN,0.0
3,220004,NaN,2022-01-03,Saizi,Nedson,48.0,M,Chilikumanda,Limbe,Caleb,...,No,NaN,NaN,Emergency,Major,ASA 3,1975.0,00:00:00,NaN,0.0
4,220005,NaN,2022-01-03,Beatrice,Hezekia,26.0,F,Mazengera,Obs/Gyn,NaN,...,No,NaN,NaN,NaN,NaN,NaN,1997.0,00:00:00,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,221537,NaN,2022-12-31,PATUMA,BWANALI,12.0,F,CHAUMA,Lam,Vitu,...,No,06:35:00,07:40:00,Urgent,Major,NaN,2011.0,01:05:00,65.0,65.0
1536,221538,NaN,2022-12-31,LUFIYA,FESTON,21.0,F,MCHEDZA,Obs/Gyn,NaN,...,No,08:15:00,09:05:00,Emergency,Intermediate,NaN,2002.0,00:50:00,50.0,50.0
1537,221539,NaN,2022-12-31,MOREEN,ZENAS,18.0,F,MITAWA,Obs/Gyn,NaN,...,No,NaN,NaN,Emergency,Intermediate,NaN,2005.0,00:00:00,NaN,0.0
1538,221540,NaN,2022-12-31,MERVIS,CHITEKWERE,8.0,F,CHITEKWERE,Vaylann,Wongani,...,No,NaN,NaN,Urgent,Intermediate,NaN,2015.0,00:00:00,NaN,0.0


In [19]:
df.columns

Index(['theatre_book', 'hospital_id', 'date_of_surgery', 'first_name',
       'last_name', 'age_years', 'sex', 'village', 'surgeon',
       'first_assistent_instructor', 'second_assistent', 'anaestesist',
       'nurse', 'anesthesia', 'department', 'indication_for_surgery',
       'surgery_type', 'final_diagnosis_category', 'final_diagnosis_free_text',
       'side', 'main_procedure_category', 'procedure_free_text', 'histology',
       'starting_time', 'finishing_time', 'urgency', 'surgery_severity',
       'asa_score', 'year_of_birth', 'operation_time_minutes',
       'op_minutes_calc', 'op_minutes_recorded'],
      dtype='object')

In [24]:
to_analyze(df, ['first_name','last_name', 'age_years', 'sex', 'village', 'surgeon'])

In [21]:
# Drop hospital_id 99% missing
df = df.drop(columns=["hospital_id"])

In [22]:
# convert date_of_surgery to datetime
df["date_of_surgery"] = pd.to_datetime(
    df["date_of_surgery"],
    errors="coerce"
)

In [25]:
# Standardize age
df["age_years"].describe()

count    1507.000000
mean       33.930325
std        18.710340
min         1.000000
25%        21.000000
50%        29.000000
75%        45.000000
max        97.000000
Name: age_years, dtype: float64

In [26]:
df["age_years"] = df["age_years"].astype("Int64")
df["age_years"].dtype
df["age_years"].isna().sum()

In [30]:
# Standardizing sex

In [29]:
df["sex"].value_counts(dropna=False)
df["sex"] = pd.Categorical(
    df["sex"],
    categories=["F", "M"]
)

In [34]:
# Cleaning surgeon
df["surgeon"].value_counts(dropna=False)

surgeon
Obs/Gyn     776
Limbe       197
Lam         167
Caleb        97
Terry        79
Widmann      57
Stuebing     48
Other        35
Vitu         34
Vaylann      32
Thoko        16
NaN           2
Name: count, dtype: int64

In [35]:
df["surgeon"] = (
    df["surgeon"]
    .str.strip()
    .str.title()
)

In [37]:
# Classify when not an individual
def classify_surgeon(x):
    if pd.isna(x):
        return "unknown"
    if x == "Obs/Gyn":
        return "specialty"
    if x in ["Limbe", "Lam"]:
        return "facility"
    if x == "Other":
        return "unknown"
    return "individual"

df["surgeon_type"] = df["surgeon"].apply(classify_surgeon)

In [39]:
df["surgeon_type"] = pd.Categorical(
    df["surgeon_type"],
    categories=["individual", "facility", "specialty", "unknown"]
)

In [40]:
# normalize names

In [41]:
for col in ["first_name", "last_name"]:
    df[col] = (
        df[col]
        .astype("string")
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)
        .str.title()
    )

In [42]:
df[["first_name", "last_name"]].isna().sum()

first_name    3
last_name     4
dtype: int64

In [43]:
# lowercase village
df["village"] = df["village"].str.lower()

In [45]:
df["village"].value_counts(dropna=False)

village
lilongwe     295
mazengera    177
dedza        165
NaN          114
tambala       58
            ... 
kde            1
zomba          1
ntchisi        1
mponera        1
mchedza        1
Name: count, Length: 414, dtype: int64

In [54]:
%pip install googlemaps


  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Installing backend dependencies ..done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40818 sha256=4c22560f0023aacc6947ff4609b98e14e5ba5f67830152c3e662802bb2d68c18
  Stored in directory: /home/mat2m10/.cache/pip/wheels/4c/6a/a7/bbc6f5c200032025ee655deb5e163ce8594fa05e67d973aad6
Successfully built googlemaps
Note: you may need to restart the kernel to use updated packages.


In [55]:
NKHOMA_HOSPITAL = (-14.04314, 34.10294)  # lat, lng

In [56]:
import os
import time
import pandas as pd
import googlemaps

# 1) Set your API key (recommended: environment variable)
# In bash: export GOOGLE_MAPS_API_KEY="YOUR_KEY"
gmaps = googlemaps.Client(key=os.environ["GOOGLE_MAPS_API_KEY"])

# 2) Build the unique location list to geocode
# Use village for now; you can swap in a different column later.
locations = (
    df["village"]
    .dropna()
    .astype("string")
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
    .drop_duplicates()
    .sort_values()
    .tolist()
)

# 3) Load / create a geocode cache (CSV)
CACHE_PATH = "geocode_cache_mw.csv"
if os.path.exists(CACHE_PATH):
    cache = pd.read_csv(CACHE_PATH)
else:
    cache = pd.DataFrame(columns=["query", "formatted_address", "lat", "lng", "place_id", "status"])

cache = cache.drop_duplicates(subset=["query"], keep="last")
cached_queries = set(cache["query"].astype(str))

def geocode_one(query: str) -> dict:
    """
    Geocode a place string, biased to Malawi.
    Returns a dict with status + fields (or blanks if not found).
    """
    q = str(query).strip()
    if not q:
        return {"query": query, "formatted_address": None, "lat": None, "lng": None, "place_id": None, "status": "EMPTY"}

    # Append country context to reduce ambiguity
    # (Still preserves your original string in `query`)
    q_mw = f"{q}, Malawi"

    try:
        # region='mw' biases results; it does not hard-restrict
        results = gmaps.geocode(q_mw, region="mw")
        if not results:
            return {"query": q, "formatted_address": None, "lat": None, "lng": None, "place_id": None, "status": "ZERO_RESULTS"}

        top = results[0]
        loc = top["geometry"]["location"]
        return {
            "query": q,
            "formatted_address": top.get("formatted_address"),
            "lat": loc.get("lat"),
            "lng": loc.get("lng"),
            "place_id": top.get("place_id"),
            "status": "OK",
        }
    except Exception as e:
        return {"query": q, "formatted_address": None, "lat": None, "lng": None, "place_id": None, "status": f"ERROR: {type(e).__name__}"}

# 4) Geocode only the missing ones (rate-limited + cached)
new_rows = []
for i, q in enumerate(locations, start=1):
    if q in cached_queries:
        continue

    row = geocode_one(q)
    new_rows.append(row)

    # gentle throttling to avoid hammering the API (and reduce errors)
    time.sleep(0.12)

# 5) Save updated cache
if new_rows:
    cache = pd.concat([cache, pd.DataFrame(new_rows)], ignore_index=True)
    cache.to_csv(CACHE_PATH, index=False)

# 6) Create a results table for this run (aligned to your unique inputs)
geo_results = cache[cache["query"].isin(locations)].copy()

# Quick audit
geo_results["status"].value_counts(dropna=False), geo_results.head()

KeyError: 'GOOGLE_MAPS_API_KEY'

In [46]:
to_analyze(df, ['village'])

In [49]:
village_audit = (
    df["village"]
    .dropna()
    .astype("string")
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
    .str.title()
    .value_counts()
    .reset_index()
)

village_audit.columns = ["village", "count"]

village_audit.shape


(413, 2)

In [53]:
village_audit.to_csv(f"{path_desktop}/village_audit.csv", index=False)


In [7]:
pkl_path = f"{base_dir}/theatre_book_database_2022_clean.pkl"
xlsx_path = f"{base_dir}/theatre_book_database_2022_clean.xlsx"

df.to_pickle(pkl_path)
df.to_excel(xlsx_path, index=False)

In [20]:
to_analyze(df, ['theatre_book', 'hospital_id', 'date_of_surgery'])